I am trying to test reinjection for different structures. this is just a general test.

the idea is to remove the acsl in combined file in a new column, and then reinject right after.

combined_file -> code_implementation -> combined_file

In [21]:
import pandas as pd
import re
import ast
from validation_treesitter import inject_acsl_specs_treesitter, extract_structure, validate_and_inject

In [22]:
df = pd.read_csv('casp_pt4_1.csv')
df.drop(columns=['unit_tests','description','skeleton_w_acsl','skeleton'], inplace=True)
df.head()

,combined_file,spec
0,/*@ assigns \nothing;\n ensures \result == ...,"[[], '/*@ assigns \\nothing;\n ensures \\re..."
1,/*@ requires n > 0;\n assigns \nothing;\n ...,"[[], '/*@ requires n > 0;\n assigns \\nothi..."
2,#include <limits.h>\n#include <string.h>\n/*@ ...,"[[""/*@ predicate matcher_ab(char * x0) = ((((..."
3,/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...,"[[], '/*@\n\trequires n>0;\n\trequires 0<=k<n;..."
4,/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....,"[[], '/*@\n\trequires n>0;\n\trequires\\valid(..."


In [24]:
def remove_all_comments(c_code: str) -> str:
    if pd.isna(c_code) or not c_code:
        return ""
    c_code = re.sub(r'/\*.*?\*/', '', c_code, flags=re.DOTALL)
    c_code = re.sub(r'//.*?$', '', c_code, flags=re.MULTILINE)
    c_code = re.sub(r'\n\s*\n+', '\n\n', c_code)
    
    return c_code.strip()

df['code_implementation'] = df['combined_file'].apply(remove_all_comments)
df.head()

,combined_file,spec,code_implementation
0,/*@ assigns \nothing;\n ensures \result == ...,"[[], '/*@ assigns \\nothing;\n ensures \\re...","int clamp(int v, int min, int max)\n{\n\tint l..."
1,/*@ requires n > 0;\n assigns \nothing;\n ...,"[[], '/*@ requires n > 0;\n assigns \\nothi...",int pick_index(int n) { return 0; }\n\nint pic...
2,#include <limits.h>\n#include <string.h>\n/*@ ...,"[[""/*@ predicate matcher_ab(char * x0) = ((((...",#include <limits.h>\n#include <string.h>\n\nin...
3,/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...,"[[], '/*@\n\trequires n>0;\n\trequires 0<=k<n;...","void q2(int a[],int n,int k)\n{\n\n\tfor (int ..."
4,/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....,"[[], '/*@\n\trequires n>0;\n\trequires\\valid(...","void bubbleinnerloop(int a[], int n)\n{\n\n\tf..."


In [25]:
row = df.iloc[1]
code_impl = row['code_implementation']

print('Initial:\n' + row['combined_file'] + '\n')
print('Stripped:\n' + code_impl + '\n')
print('Reinjected:\n' + inject_acsl_specs_treesitter(ast.literal_eval(row['spec']), code_impl) + '\n')


Initial:
/*@ requires n > 0;
    assigns \nothing;
    ensures 0 <= \result && \result < n;
*/
int pick_index(int n) { return 0; }

/*@ requires n > 0 && \valid(p+ (0..n-1));
    assigns \nothing;
*/
int pick_element(int* p, int n) {
  int i = pick_index(n);
  
  return p[i];
}

/*@
  requires \valid(p);
  assigns \nothing;
  ensures \result == p[0];
*/
int pick_first(int* p) {
  return p[0];
}

Stripped:
int pick_index(int n) { return 0; }

int pick_element(int* p, int n) {
  int i = pick_index(n);

  return p[i];
}

int pick_first(int* p) {
  return p[0];
}

Reinjected:
/*@ requires n > 0;
    assigns \nothing;
    ensures 0 <= \result && \result < n;
*/
int pick_index(int n) { return 0; }

/*@ requires n > 0 && \valid(p+ (0..n-1));
    assigns \nothing;
*/
int pick_element(int* p, int n) {
  int i = pick_index(n);

  return p[i];
}

/*@
  requires \valid(p);
  assigns \nothing;
  ensures \result == p[0];
*/
int pick_first(int* p) {
  return p[0];
}

